## Coursera Capstone Project - Week 1 & Week 2
### Customer Segmentation and Clustering based on Location Amenities

In [2]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course")

Hello Capstone Project Course


In [24]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=e99446f77502fa30499e481cd5a3e2f088a1c7fa64b3b41e1ce2ffcf8fda277a
  Stored in directory: c:\users\visha\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install tabulate

  Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23274 sha256=21b15d871f286de389368a19bc5d752f800fef9ff52ecab620e018a3cd9c56ee
  Stored in directory: c:\users\visha\appdata\local\pip\cache\wheels\86\8c\d9\3b6c2f55efaefe656c52b725d2fdc5669085c83444979db8b3
Successfully built tabulate
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.8.2-py3-none-any.whl (106 kB)
  Using cached soupsieve-2.0-py2.py3-none-any.whl (32 kB)



In [3]:
pip install requests

  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.25.8-py2.py3-none-any.whl (125 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)



In [65]:
pip install csv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


# Data Science Problem Definition
To identify and explore features of Manhattan and Torronto, to come up the conclusions where are the better business prospects in terms of opening restaurant. Unerstanding which different stores are available in neighbourhoods. We want to explore how much they are similar or dissimilar in aspects from a tourist point of view regarding food, accommodation and culture. Based on this information suggestion is to open the restaurant where there is less competition among similar group shops or restaurants in doing the business.

# Data Description

Using Foursquare API - explore neighbourhoods data of Toronto and Manhattan cities.
Focus on the data releven to veneus related restaurants, eateries, coffee shops and entertainment venues. Analyze this information to decide on which city is more suitable for opening the restaurant considering different food items and based on different types of visitors and tourist. 


# Methodology

For Manhattan data - we will use the saved file which is already explored through Foursqueare API, in this we had sorted top venues and categories.

Using similar approach we will be deriving top venues for Toronto.

### 1. Webscrapping the Borough and its Neighbrourhood 

### 2. Transforming this data in Post Code, Borough, Neighbourhood, Latitude & Longitude 

### 3. With the help of FourSquare API and Geo Code data from previous step derive neighbourhood    information related to venues based on different categories.

### 4. Visualizing Top Locations based on categories 

### 5. Draw conculusion based on Visualizations and provide recommendation to start new business

### Intoduction
In this Project we will be webscrapping the data for Torronto and its neighbourhood. Derive Latitude and Longitudes for all the neighbourhood locations. Explore to get common venue categories in each neighbourhood, and use the features to group the neighbourhood into clusters.

We have already analysed the New York and Manhattan top 10 locations, we will be using New York data with Torronto to find top 10 locations of Torronto.

### 1. Webscrapping the Borough and its Neighbrourhood data

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
# Import libraries

import pandas as pd
import numpy as np
import requests
import csv
from bs4 import BeautifulSoup

# Read source URL & Parse using lxml
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

CSV_FILE_NAME = 'cms_scrape.csv'

# Open CSV File
csv_file = open(CSV_FILE_NAME,'w',newline='')

csv_writer = csv.writer(csv_file)
# print(type(csv_writer))

# Write header column names to the file
csv_writer.writerow(['Postal Code', 'Borough', 'Neighbourhood'])

# Loop through table rows
for tr in soup.find_all('tr'):
    
## For each row traverse to each columns    
    for td in tr.find_all('td'):
        
## Read Postal Code        
        try:
            post_code = td.p.b.text                    
        except Exception as e:
            post_code = None
#         print('Postal Code :', post_code)    

## Read Borough
        try:
            borough = td.p.span.a.text
        except Exception as e:
            borough = None
#         print('Borough :', borough)
        
## Read Neighbourhoood    
        try:
            first_link = td.p.span.a            
            neighbour = []
            for a in first_link.find_next_siblings("a"):
                if a.text:
                  neighbour.append(a.text)          
        except Exception as e:
            neighbour = None
            
## Seprate neighbourhood with comma for multiple values        
        try:            
            neighbour_str = ', '.join([str(x) for x in neighbour])
        except Exception as e:
            neighbour_str = None
        
#         print('Neighbour :',neighbour_str)        

## Write the row record to file
        csv_writer.writerow([str(post_code), str(borough), str(neighbour_str)])

csv_file.close()    


#### Read the data from CSV File to Dataframe

In [4]:
df = pd.read_csv(CSV_FILE_NAME)
df.shape

(245, 3)

In [5]:
df = df[df['Borough']!= 'None']
df.shape

(101, 3)

In [6]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [7]:
# Setting the Neighbourhood values with Borough where Neighbourhood is Not Assigned

df.isna().sum()

Postal Code       0
Borough           0
Neighbourhood    13
dtype: int64

In [8]:
df['Neighbourhood'].fillna(df['Borough'], inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [9]:
df_new = df[df['Postal Code'].isin(['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]
df_new.reset_index(inplace=True, drop=True)
df_new

,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M1B,Scarborough,"Malvern, Rouge"
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M4G,East York,Leaside
4,M5G,Downtown Toronto,Bay Street
5,M2H,North York,Hillcrest Village
6,M1J,Scarborough,Scarborough Village
7,M9L,North York,Humber Summit
8,M1R,Scarborough,"Wexford, Maryvale"
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


## Geo Code Data for Postal Codes

In [10]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

df_geo_codes = pd.read_csv('Geospatial_Coordinates.csv')
df_geo_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 2. Transforming this data in Post Code, Borough, Neighbourhood, Latitude & Longitude 

In [11]:
df_geo = df.join(df_geo_codes.set_index('Postal Code'),on='Postal Code')
df_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [12]:
df_geo_new = df_new.join(df_geo_codes.set_index('Postal Code'),on='Postal Code')
df_geo_new.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M4G,East York,Leaside,43.709060,-79.363452
4,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383


In [13]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


### 3. With the help of FourSquare API and Geo Code data from previous step derive neighbourhood    information related to venues based on different categories.


In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [16]:
df_neighborhoods = df_geo[df['Borough']=='Downtown Toronto']
df_neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383


In [17]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Torronto City are 43.6541737, -79.38081164513409.


### 4. Visualizing Top Locations based on categories 

In [18]:
print(len(df_neighborhoods))
# create map of Downtown Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# # add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)    
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

18


In [19]:
CLIENT_ID = 'BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR' # your Foursquare ID
CLIENT_SECRET = 'ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR
CLIENT_SECRET:ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR


In [20]:
df_toronto = df_neighborhoods
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383


In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_geo['Borough'].unique()),
        df_geo.shape[0]
    )
)

The dataframe has 25 boroughs and 101 neighborhoods.


In [22]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383


In [23]:
# df_toronto.loc[4,'Neighbourhood']
# df_toronto.loc[4,'Latitude']

In [24]:
neighborhood_latitude = df_toronto.loc[4,'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[4, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [25]:
# type your answer here
LIMIT = 100
radius = 500
# search_query = 'Marble Hill'
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR&client_secret=ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
# results

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
# venues = results['response']['groups'][0]['items']
venues = results['response']['groups'][0]['items']
# print("Venues : ",venues)

nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues
# print("Near by Venues : ", nearby_venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
# nearby_venues['venue.categories']
nearby_venues
# print("Near by Venues : ", nearby_venues)

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues['venue.categories'].unique()

# print("Near by Venues (Unique): ", nearby_venues)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
# nearby_venues.columns

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues.head()

47 venues were returned by Foursquare.


<ipython-input-28-691d71a51137>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#     print(nearby_veunues.head())
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# type your answer here
# print(df_toronto)
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'])
# toronto_venues

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Downtown Toronto
Bay Street
Downtown Toronto
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
Downtown Toronto
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(1274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,82,82,82,82,82,82
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",18,18,18,18,18,18
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,168,168,168,168,168,168
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",100,100,100,100,100,100
"Kensington Market, Chinatown, Grange Park",87,87,87,87,87,87


In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


## Analyze Each Neighbourhood

In [35]:
# # Preprocessing
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")

toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']

# # move neighborhood column to the first column
# list(toronto_onehot.columns)

toronto_onehot.columns[-1]

# list(toronto_onehot.columns[:-1])

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
# fixed_columns

toronto_onehot = toronto_onehot[fixed_columns]

# print(fixed_columns)
toronto_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

In [36]:
toronto_onehot.shape

(1274, 207)

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(16, 207)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.012195,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.012195,0.000000,0.0,0.000000,0.0,0.024390,0.00,0.036585,0.000000,0.000000,0.024390,0.0,0.000000,0.000000,0.00000,0.024390,0.0,0.000000,0.000000,0.00000,0.170732,0.0,0.0,0.0,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.012195,0.012195,0.012195,0.000000,0.000000,0.0,0.012195,0.0,0.000000,0.0,0.0,0.000000,0.0,0.012195,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.012195,0.012195,0.012195,0.0,0.000000,0.012195,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.02439,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.012195,0.0,0.0,0.0,0.036585,0.012195,0.000000,0.048780,0.036585,0.000000,0.0,0.024390,0.012195,0.012195,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.036585,0.012195,0.012195,0.000000,0.00,0.000000,0.

### Lets print top 5 for most common venues

In [38]:
num_top_venues = 5

# toronto_grouped[toronto_grouped['Neighborhoods']=='Downtown Toronto'] #.T.reset_index()
# toronto_grouped
for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()    
    temp.columns = ['venue','freq']    
    temp = temp.iloc[1:]    
    temp['freq'] = temp['freq'].astype(float)    
    temp = temp.round({'freq': 2})    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.05
2       Ice Cream Shop  0.04
3       Sandwich Place  0.04
4  Japanese Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3   Harbor / Marina  0.06
4               Bar  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2              Gay Bar  0.04
3           Restaurant  0.03
4     Sushi Restaurant  0.03


----Commerce Court, Victoria Hotel----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.07
2   Restaurant  0.07
3        Hotel  0.05
4          Gym  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.05
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4            Be

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Juice Bar,Thai Restaurant,Department Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Rental Car Location,Boat or Ferry,Coffee Shop,Sculpture Garden,Bar
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Bubble Tea Shop,Hotel,Café,Yoga Studio,Mediterranean Restaurant
3,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Italian Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant
4,Downtown Toronto,Coffee Shop,Café,Restaurant,Seafood Restaurant,Park,Beer Bar,Cheese Shop,Bakery,Japanese Restaurant,Hotel


In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 2, 1, 1, 1, 2, 1, 2, 1])

In [51]:
# add clustering labels
# df_toronto.head()
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto
# print(toronto_merged.head(2))
toronto_merged.rename(columns={'Neighbourhood' : 'Neighborhoods'}, inplace=True)
# print(" ")
# print(toronto_merged.head(2))
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

toronto_merged.head() # check the last columns!

C:\Users\visha\projects\Coursera_Capstone\env\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Postal Code,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Restaurant,Café,Theater,Event Space,Chocolate Shop
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Restaurant,Pizza Place,Bookstore
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Cosmetics Shop,Bakery,Beer Bar,Clothing Store
31,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,Park,Beer Bar,Cheese Shop,Bakery,Japanese Restaurant,Hotel
40,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Juice Bar,Thai Restaurant,Department Store


### 5. Draw conculusion based on Visualizations and provide recommendation to start new business

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhoods'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
147,Downtown Toronto,0,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Restaurant,Café,Theater,Event Space,Chocolate Shop
31,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,Park,Beer Bar,Cheese Shop,Bakery,Japanese Restaurant,Hotel
40,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Juice Bar,Thai Restaurant,Department Store
41,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,Park,Beer Bar,Cheese Shop,Bakery,Japanese Restaurant,Hotel
58,Downtown Toronto,1,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Restaurant,Sporting Goods Shop,Scenic Lookout,Fried Chicken Joint,Brewery
67,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Bar,Gastropub
76,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Italian Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant
148,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,Park,Beer Bar,Cheese Shop,Bakery,Japanese Restaurant,Hotel
157,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gym,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Hotel,American Restaurant,Steakhouse


In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Restaurant,Pizza Place,Bookstore
22,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Breakfast Spot,Cosmetics Shop,Bakery,Beer Bar,Clothing Store
49,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Steakhouse,Lounge,Clothing Store,Bakery,Pizza Place
130,Downtown Toronto,2,Bar,Café,Vietnamese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Dumpling Restaurant,Comfort Food Restaurant,Chinese Restaurant
156,Downtown Toronto,2,Coffee Shop,Restaurant,Pub,Italian Restaurant,Bakery,Café,Pizza Place,Caribbean Restaurant,Gastropub,Breakfast Spot
165,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Bubble Tea Shop,Hotel,Café,Yoga Studio,Mediterranean Restaurant


In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
139,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Rental Car Location,Boat or Ferry,Coffee Shop,Sculpture Garden,Bar


In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,Downtown Toronto,4,Café,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Restaurant,Yoga Studio,Sushi Restaurant,Sandwich Place


### Using saved Manhattan data to compare with Toronto data

In [85]:
manhattan_data = pd.read_csv('Manhattan.csv')
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [86]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [87]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [88]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655078, -73.91065966.


In [89]:
# type your answer here
LIMIT = 100
radius = 500
# search_query = 'Marble Hill'
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=BEB5GVIMF2BYX5AMXZCA1VLZCAEJMDPS113XJVPME1V0YLUR&client_secret=ZUZHM3S0U2FWT3VX0SW5PSUUTHAYUKI5NIZW1FLKY4AOGSPR&v=20180605&ll=40.87655078,-73.91065966&radius=500&limit=100'

In [90]:
results = requests.get(url).json()
# results

In [91]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-91-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [92]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [93]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,94,94,94,94,94,94
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [94]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scan

In [95]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scan

In [96]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.07
1    Coffee Shop  0.06
2          Hotel  0.05
3  Memorial Site  0.03
4  Boat or Ferry  0.03


----Carnegie Hill----
                 venue  freq
0          Pizza Place  0.06
1          Coffee Shop  0.05
2                 Café  0.05
3  Japanese Restaurant  0.03
4               Bakery  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1  American Restaurant  0.05
2                  Bar  0.05
3   Seafood Restaurant  0.05
4    French Restaurant  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.07
1   Italian Restaurant  0.05
2               Bakery  0.05
3       Ice Cream Shop  0.03
4  American Restaurant  0.03


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1           Cocktail Bar  0.05
2  Vietnamese Restaurant  0.04
3    American Restaurant  0.04
4           Optical Shop  0.03


----Civic Center----
                  

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Gym,Women's Store,Boat or Ferry,Shopping Mall,Wine Shop,Clothing Store
1,Carnegie Hill,Pizza Place,Café,Coffee Shop,Yoga Studio,Bakery,Bookstore,Cosmetics Shop,French Restaurant,Grocery Store,Gym
2,Central Harlem,African Restaurant,French Restaurant,American Restaurant,Bar,Seafood Restaurant,Chinese Restaurant,Spa,Gym / Fitness Center,Gym,Market
3,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Hotel,Wine Shop,Ice Cream Shop,American Restaurant,Japanese Restaurant,Theater,Cosmetics Shop
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Hotpot Restaurant,Spa,Bakery,Salon / Barbershop,Bar


In [98]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 1, 1, 1, 1, 3, 0, 0])

In [99]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Pharmacy,Seafood Restaurant,Tennis Stadium,Supplement Shop,Miscellaneous Shop,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Hotpot Restaurant,Spa,Bakery,Salon / Barbershop,Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Grocery Store,Bakery,Chinese Restaurant,Mobile Phone Shop,Bank,Mexican Restaurant,Sandwich Place,Supermarket,Supplement Shop
3,Manhattan,Inwood,40.867684,-73.921210,3,Café,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Park,Frozen Yogurt Shop,Deli / Bodega,Bakery,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Deli / Bodega,Coffee Shop,Mexican Restaurant,Café,Sandwich Place,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio


In [100]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,African Restaurant,French Restaurant,American Restaurant,Bar,Seafood Restaurant,Chinese Restaurant,Spa,Gym / Fitness Center,Gym,Market
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Pizza Place,Deli / Bodega,Japanese Restaurant,Wine Shop,Diner,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Café,Gym / Fitness Center,Gym,Burger Joint,Sporting Goods Shop
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Café,Coffee Shop,Bakery,Mediterranean Restaurant,Indian Restaurant,Yoga Studio,Gym / Fitness Center
13,Lincoln Square,Theater,Plaza,Café,Italian Restaurant,Concert Hall,Performing Arts Venue,Indie Movie Theater,American Restaurant,Gym / Fitness Center,French Restaurant
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,French Restaurant,Indian Restaurant,Cosmetics Shop,Caribbean Restaurant,Boutique,Seafood Restaurant
19,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Pizza Place,Cocktail Bar,Vegetarian / Vegan Restaurant,Korean Restaurant,Chinese Restaurant,Italian Restaurant
25,Manhattan Valley,Indian Restaurant,Coffee Shop,Bar,Pizza Place,Yoga Studio,Mexican Restaurant,Thai Restaurant,Playground,Park,Peruvian Restaurant
27,Gramercy,Italian Restaurant,Bar,American Restaurant,Bagel Shop,Mexican Restaurant,Pizza Place,Spa,Diner,Cocktail Bar,Ice Cream Shop
29,Financial District,Coffee Shop,American Restaurant,Bar,Gym,Pizza Place,Italian Restaurant,Wine Shop,Hotel,Gym / Fitness Center,Cocktail Bar


In [102]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Hotpot Restaurant,Spa,Bakery,Salon / Barbershop,Bar
8,Upper East Side,Coffee Shop,Italian Restaurant,Exhibit,Art Gallery,Bakery,Juice Bar,Gym / Fitness Center,Pizza Place,Hotel,French Restaurant
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,American Restaurant,Coffee Shop,Hotel,Spa,Wine Shop,Sandwich Place,Food Court
15,Midtown,Hotel,Coffee Shop,Clothing Store,Steakhouse,Theater,Bakery,Cocktail Bar,Sporting Goods Shop,Bookstore,Café
16,Murray Hill,Sandwich Place,Coffee Shop,Japanese Restaurant,American Restaurant,Hotel,Mediterranean Restaurant,Italian Restaurant,Gym,Gym / Fitness Center,Vietnamese Restaurant
17,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Hotel,Wine Shop,Ice Cream Shop,American Restaurant,Japanese Restaurant,Theater,Cosmetics Shop
20,Lower East Side,Pizza Place,Coffee Shop,Art Gallery,Chinese Restaurant,Café,Japanese Restaurant,Cocktail Bar,Theater,Ramen Restaurant,Bakery
21,Tribeca,American Restaurant,Italian Restaurant,Park,Café,Spa,Greek Restaurant,Coffee Shop,Wine Bar,Wine Shop,Boutique
22,Little Italy,Café,Bakery,Bubble Tea Shop,Cocktail Bar,Sandwich Place,Salon / Barbershop,Mediterranean Restaurant,Italian Restaurant,Yoga Studio,Women's Store
23,Soho,Clothing Store,Boutique,Shoe Store,Women's Store,Art Gallery,Bakery,Sporting Goods Shop,Mediterranean Restaurant,Men's Store,Yoga Studio


In [103]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Boat or Ferry,Park,Cocktail Bar,Baseball Field,Coffee Shop,Gym / Fitness Center,Heliport,Farmers Market,Pet Service


In [104]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Grocery Store,Bakery,Chinese Restaurant,Mobile Phone Shop,Bank,Mexican Restaurant,Sandwich Place,Supermarket,Supplement Shop
3,Inwood,Café,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Park,Frozen Yogurt Shop,Deli / Bodega,Bakery,Chinese Restaurant
4,Hamilton Heights,Pizza Place,Deli / Bodega,Coffee Shop,Mexican Restaurant,Café,Sandwich Place,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio
5,Manhattanville,Coffee Shop,Seafood Restaurant,Italian Restaurant,Mexican Restaurant,Park,Bus Stop,Chinese Restaurant,Indian Restaurant,Café,Grocery Store
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Pizza Place,Deli / Bodega,Café,Beer Bar,Taco Place,Gas Station
11,Roosevelt Island,Coffee Shop,Sandwich Place,Japanese Restaurant,Kosher Restaurant,Soccer Field,Farmers Market,Supermarket,Metro Station,Outdoors & Recreation,School
26,Morningside Heights,Bookstore,Park,Coffee Shop,American Restaurant,Burger Joint,Sandwich Place,New American Restaurant,Deli / Bodega,Ice Cream Shop,Mexican Restaurant
36,Tudor City,Café,Park,Mexican Restaurant,Deli / Bodega,Diner,Pizza Place,Coffee Shop,Dog Run,Gym,Spanish Restaurant


In [105]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Pharmacy,Seafood Restaurant,Tennis Stadium,Supplement Shop,Miscellaneous Shop,Donut Shop


## Results and Conclusion

On examining the neighbourhood clusters of Toronto and Manhattan - there are more restaurants in Manhattan when compared to Toronto.

It is suggested to open restaurant or other such business in Downtown Toronto as there will be less competetion. Manhattan has more number of cafes and different types of restaurants compared to Downtown Toronto.

To start new business veture in terms of opening new food chain or restaurants, it can be concluded Toronto or its neighbourhood is more suitable.